In [2]:
model_name = "bagging"
random_state = 42
bayesian_search = True

print(f"model_name: {model_name}")
print(f"bayesian_search: {bayesian_search}")

model_name: xgboost
randomized_grid_search: True
canonical_grid_search: False


In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    dir = "/storage/"
else:
    dir = "/content/drive/MyDrive/Colab/gscreen/"

print(f"Data dir: {dir}")

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import time

from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn import tree
from sklearn import ensemble

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

print(f"Numpy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

In [ ]:
try:
  from skopt import BayesSearchCV
  print(f"The scikit-optimize package exist.")
except Exception as e:
  print(f"Import scikit-optimize error: {e}")
  print(f"Try to re-install scikit-optimize package")
  ! pip install scikit-optimize

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

print (f"{sys.modules[BayesSearchCV.__module__[:BayesSearchCV.__module__.index('.')]].__version__}")

In [7]:
#%% Load data ------------------------------------------------------------------
with open(dir + f"X_fit.joblib.compressed", "rb") as f:
  X_fit = joblib.load(f)

with open(dir + f"X_train.joblib.compressed", "rb") as f:
  X_train = joblib.load(f)

with open(dir + f"X_val.joblib.compressed", "rb") as f:
  X_val = joblib.load(f)

print(f"X_fit: {X_fit.shape}, {type(X_fit)}\
\nX_train: {X_train.shape}, {type(X_train)}\
\nX_val: {X_val.shape}, {type(X_val)}\n")

y_fit = joblib.load(dir + f"y_fit.joblib")
y_train = joblib.load(dir + f"y_train.joblib")
y_val = joblib.load(dir + f"y_val.joblib")

print(f"y_fit: {y_fit.size}, {type(y_fit)}\
\ny_train: {y_train.size}, {type(y_train)}\
\ny_val: {y_val.size}, {type(y_val)}")

X_fit: (296721, 279)
X_train: (296727, 279)
X_val: (5000, 279)

y_fit: 296721
y_train: 296727
y_val: 5000


In [3]:
#%% Define model parameters for starting tuning
model_params = {
    "base_estimator": tree.ExtraTreeRegressor(
        criterion="mse",  # {"mse", "friedman_mse", ""mae"} default="mse"
        splitter="random",  # {"random", "best"} default="random"
        max_depth=None,  # default=None
        min_samples_split=2,  # default=2
        min_samples_leaf=1,  # default=1
        random_state=random_state,
    ),
    "n_estimators": 80,
    "max_samples": 1.0,
    "max_features": 1.0,
    "bootstrap": True,
    "bootstrap_features": False,
    "oob_score": False,
    "n_jobs": None,
    "random_state": random_state,
    "verbose": 3,
}
model = ensemble.BaggingRegressor(**model_params)

In [ ]:
#%% ----------------------- Bayesian Optimization ------------------------------
# Their core idea of Bayesian Optimization is simple:
# when a region of the space turns out to be good, it should be explored more.
# Real: Continuous hyperparameter space.
# Integer: Discrete hyperparameter space.
# Categorical: Categorical hyperparameter space.
bayes_space = {
    # "n_estimators": Integer(2000, 3000),
    "subsample": Real(0.6, 1.0),
    "colsample_bytree": Real(0.7, 1.0),
    "max_depth": Integer(3, 20),
    "min_child_weight": Integer(1, 20),
    "learning_rate": Real(0.01, 0.4),
    "reg_lambda": Real(0.5, 5),
    "gamma": Real(0.0, 5),
}

In [5]:
def accuracy(real_rates, predicted_rates):
    """Project's accuracy value estimator"""
    return np.average(abs(real_rates / predicted_rates - 1.0)) * 100.0

def calc_metrics(model, X, y):
    """Calculates result metrics"""

    from sklearn.metrics import max_error
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import mean_squared_error

    y_true = y
    y_pred = model.predict(X)

    me = max_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mare = accuracy(y_true, y_pred)

    print(f"\tMax Error: {me}")
    print(f"\tMean Absolute Error: {mae}")
    print(f"\tRoot Mean Squared Error: {rmse}")
    print(f"\tMean Absolute Ratio Error: {mare}")

In [6]:
#%% Define custom scorer to evaluate basic models
scorer = make_scorer(
    score_func=accuracy,
    greater_is_better=True,  # Whether score_func is a score function (default),
    # meaning high is good, or a loss function, meaning low is good.
)
#%% Define Cross Validation parameters for grid search
# Define classic cross validation method and params
cv = model_selection.RepeatedKFold(
    n_splits=3,  #! Dont forget
    n_repeats=1,
    random_state=random_state,
)

In [ ]:
#%% ---------------------- RandomizedSearchCV ----------------------------------
rs_bp, rs_be = {}, {}
if randomized_grid_search:
    tic = time.time()
    # Define RANDOMIZED grid search
    random_search = model_selection.RandomizedSearchCV(
        model,
        param_distributions=param_dist,
        n_iter=30,  #! default=10 Number of parameter settings that are sampled.
        scoring=scorer,
        n_jobs=None,
        cv=cv,
        refit=True,
        return_train_score=True,
        verbose=8,
        random_state=random_state,
    )
    # Make RANDOMIZED grid search
    model_random_search = random_search.fit(
        X_fit,
        y_fit,
    )
    # Print out best parameters
    rs_bp = model_random_search.best_params_
    rs_be = model_random_search.best_estimator_
    print(f"\nRandomized search:\nBest params are:\n {rs_bp}")
    print(f"\nBest estimator is:\n {rs_be}")

    min, sec = divmod(time.time() - tic, 60)
    print(f"\nRandomized grid search taken: {int(min)}min {int(sec)}sec")

    # Print out results
    print(f"{model_name.title()} random search:")
    print("TRAIN set:")
    calc_metrics(model_random_search, X=X_train, y=y_train)
    print("VALIDATION set:")
    calc_metrics(model_random_search, X=X_val, y=y_val)

    model = model_random_search.best_estimator_

Fitting 4 folds for each of 30 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
#%% --------------------------- GridSearchCV -----------------------------------
#! Dont forget to fine set up the Grid Search parameters in algo_*.py module
#! and reload that module
gs_bp, gs_be = {}, {}
if canonical_grid_search:
    tic = time.time()
    # Define SIMPLE grid search
    grid_search = model_selection.GridSearchCV(
        model,
        param_grid=param_grid,
        scoring=scorer,
        n_jobs=None,  # Number of jobs to run in parallel. -1 = using all processors
        cv=cv,  # Determines the cross-validation splitting strategy.
        refit=True,  # Refit model with the best found parameters on the whole dataset.
        return_train_score=True,
        verbose=8,  # Controls the verbosity: the higher, the more messages.
    )
    # Make SIMPLE grid search
    model_grid_search = grid_search.fit(
        X_fit,
        y_fit,
    )
    # Print out best parameters
    gs_bp = model_grid_search.best_params_
    print(f"Grid search:\nBest params are:\n {gs_bp}")

    min, sec = divmod(time.time() - tic, 60)
    print(f"\nGrid search taken: {int(min)}min {int(sec)}sec")

    # Print out results
    print(f"{model_name.title()} grid search:")
    print("TRAIN set:")
    calc_metrics(model_grid_search, X=X_train, y=y_train)
    print("VALIDATION set:")
    calc_metrics(model_grid_search, X=X_val, y=y_val)

    model = model_grid_search.best_estimator_
    print(f"{model}")

In [ ]:
#%% ----------------------- Bayesian Optimization ------------------------------
bs_bp, bs_be = {}, {}
if bayesian_search:
    tic = time.time()
    # Define bayesian search
    bayes_search = BayesSearchCV(
        model,
        search_spaces=bayes_space,
        n_iter=50,  #! default=50 Number of parameter settings that are sampled.
        scoring=scorer,
        n_jobs=None,  # Number of jobs to run in parallel. -1 = using all processors
        cv=3,  # default 3-fold cross validation.
        refit=True,
        verbose=3,
        return_train_score=True,
        random_state=random_state,
    )
    # Make search
    model_bayes_search = bayes_search.fit(
        X_fit,
        y_fit,
    )
    # Print out best parameters
    bs_bp = model_bayes_search.best_params_
    print(f"Bayesian search:\nBest params are:\n {bs_bp}")

    min, sec = divmod(time.time() - tic, 60)
    print(f"\nBayesian search taken: {int(min)}min {int(sec)}sec")

    # Print out results
    print(f"{model_name.title()} Bayesian search:")
    print("TRAIN set:")
    calc_metrics(model_bayes_search, X=X_train, y=y_train)
    print("VALIDATION set:")
    calc_metrics(model_bayes_search, X=X_val, y=y_val)

    model = model_bayes_search.best_estimator_
    print(f"{model}")

In [ ]:
#%% Fit the model w\o parameters searching -------------------------------------
if not randomized_grid_search and not canonical_grid_search and not bayesian_search:
    # Train the model
    tic = time.time()
    model.fit(X_fit, y_fit)
    # Evaluate time spent
    min, sec = divmod(time.time() - tic, 60)
    print(f"Time taken: {int(min)}min {int(sec)}sec")
    print(f"{model}\n")

    # Print out results
    print(f"{model_name.title()} model:")
    print("TRAIN set:")
    calc_metrics(model, X=X_train, y=y_train)
    print("VALIDATION set:")
    calc_metrics(model, X=X_val, y=y_val)